# 947. Most Stones Removed with Same Row or Column
https://leetcode.com/problems/most-stones-removed-with-same-row-or-column/description/?envType=daily-question&envId=2024-08-29

In [8]:
# DFS
from collections import defaultdict

def removeStones(stones):
    def dfs(x, y):
        visited.add((x, y))
        for next_y in rows[x]:
            if (x, next_y) not in visited:
                dfs(x, next_y)
        for next_x in cols[y]:
            if (next_x, y) not in visited:
                dfs(next_x, y)

    
    rows = defaultdict(list)
    cols = defaultdict(list)
    
    for x, y in stones:
        rows[x].append(y)
        cols[y].append(x)

    print(rows,cols)
    visited = set()
    num_of_islands = 0
    
    for x, y in stones:
        if (x, y) not in visited:
            dfs(x, y)
            num_of_islands += 1
    print(visited,num_of_islands)
    return len(stones) - num_of_islands

stones1 = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
removeStones(stones1)

defaultdict(<class 'list'>, {0: [0, 1], 1: [0, 2], 2: [1, 2]}) defaultdict(<class 'list'>, {0: [0, 1], 1: [0, 2], 2: [1, 2]})
{(0, 1), (1, 2), (0, 0), (2, 1), (2, 2), (1, 0)} 1


5

這個問題可以透過將石頭視為一個圖中的節點來解決，其中每個石頭與其他石頭連接在同一行或同一列上。解決的關鍵在於找到圖中的連通分量的數量，並計算可以移除的石頭數量，這相當於圖中節點數減去連通分量的數量。 

## 問題分析 
1. 圖的表示：我們將每塊石頭看作圖中的一個節點。如果兩塊石頭在同一行或同一列上，那麼它們之間有一條邊連接。這意味著石頭可以透過共享行或列與其他石頭形成連通分量。 
2. 計算可移除的石頭數：如果一組石頭構成一個連通分量，則最多可以移除 component_size - 1 塊石頭（因為最終只剩下其中一塊）。總共可以移除的石頭數就是所有連通分量中可以移除的石頭數的總和。 
3. 使用並查集 (Union-Find)：這是解決此類連通分量問題的有效方法。

In [ ]:
# GPT : Union-Find
class UnionFind:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        
        if rootX != rootY:
            if self.rank[rootX] > self.rank[rootY]:
                self.parent[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.parent[rootX] = rootY
            else:
                self.parent[rootY] = rootX
                self.rank[rootX] += 1

    def add(self, x):
        if x not in self.parent:
            self.parent[x] = x
            self.rank[x] = 0

def removeStones(stones):
    uf = UnionFind()

    for x, y in stones:
        uf.add(x)
        uf.add(~y)  # 使用 ~y 将 y 映射到一个独特的整数表示，避免冲突
        uf.union(x, ~y)

    # 统计不同的连通分量数量
    unique_roots = len({uf.find(x) for x, y in stones} | {uf.find(~y) for x, y in stones})
    
    return len(stones) - unique_roots

# 示例输入
stones1 = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
print(removeStones(stones1))  # 输出：5

stones2 = [[0,0],[0,2],[1,1],[2,0],[2,2]]
print(removeStones(stones2))  # 输出：3

stones3 = [[0,0]]
print(removeStones(stones3))  # 输出：0


## 解釋 
1. UnionFind 資料結構：UnionFind 透過 parent 陣列追蹤每個節點的父節點，透過 rank 陣列追蹤樹的高度以實現路徑壓縮優化。
2. 新增與合併節點：在 removeStones 函數中，我們首先將所有的石頭座標 (x, y) 加入並查集中，並將行與列座標連接在一起。這是透過將列座標 y 映射為負數 ~y 來實現的，以避免與行座標衝突。 
3. 計算獨立連通分量數：最終我們透過 find 函數確定所有節點的根，並計算出不同的根的數量，即連通分量的數量。可以移除的石頭數量等於 len(stones) - 連通分量數。 
## 複雜度分析 
* 時間複雜度：O(n)，其中 n 是石頭的數量。因為並查集的每個操作的平均時間複雜度為 O(1)。 
* 空間複雜度：O(n)，需要儲存每個節點的父節點和秩資訊。